## Analytics Goal : Artist-level analysis. Looking at the Popular Artists, Popular songs, Artist Sound Quality, similar artists and  tags

In [1]:
import numpy as np
import pyspark
import re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
63,application_1638946947785_0065,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%local
import os
import re
from IPython.display import display
from ipywidgets import widgets, interactive
import ipywidgets as widgets
%config InlineBackend.figure_format = 'retina'
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import time
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
t_start = time.time()

In [ ]:
# %%configure -f 
# {
# "conf":{
#         "spark.pyspark.python": "python3",
#         "spark.pyspark.virtualenv.enabled": "true",
#         "spark.pyspark.virtualenv.type":"native",
#         "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
    
#         "spark.executor.heartbeatInterval":"10800s",
#         "spark.network.timeout":"24h",
    
#         "spark.driver.memory": "10G",
#         "spark.executor.memory": "7G",
#         "spark.executor.cores":"2",

#         "spark.app.name":"msds694_16_project"
#       }
# }

In [3]:
ct_partitions = 8

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
input_rdd = sc.textFile("s3://millionsongs-data/new_g4g.csv")
table = input_rdd.map(lambda x: x.split('\t'))
header = table.take(1)
table_body = table.filter(lambda x : x != header[0]).cache()
column_index = {header[0][i]:i for i in range(len(header[0]))}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Hottest Artists

In [5]:
artist_rdd = table_body.filter(lambda x : x[column_index['get_artist_familiarity']]!= '').filter(lambda x : 
                        x[column_index['get_artist_hotttnesss']]!= '').map(lambda x: 
                       (x[column_index['get_artist_id']],
                        [x[column_index['get_artist_name']],
                        float(x[column_index['get_artist_familiarity']]),
                        float(x[column_index['get_artist_hotttnesss']]),
                        x[column_index['get_artist_mbtags']],
                        x[column_index['get_artist_mbtags_count']],
                        x[column_index['get_artist_terms']],
                        x[column_index['get_artist_terms_freq']],
                        x[column_index['get_similar_artists']],
                        x[column_index['get_energy']]])).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
## Ineficient code
# artist_rdd_names = artist_rdd.map(lambda x: (x[1][0], 
#                     x[1][2])).groupByKey().mapValues(
#     lambda x: sum(x)/len(x)).sortBy(lambda x :x[1], ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
artist_rdd_names = artist_rdd.map(lambda x: (x[1][0], 
                    x[1][2])).mapValues(lambda x: (x, 1))\
                             .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))\
                             .mapValues(lambda x: x[0]/x[1])\
                             .sortBy(lambda x :x[1], ascending=False, numPartitions = ct_partitions)

hottest_artists = artist_rdd_names.map(lambda x: (x[0],x[1])).filter(lambda x : x[1] > 0.8)
hottest_artists_name = hottest_artists.map(lambda x: x[0]).take(100) ## saving the list of top 20 artists for later filtering purposes
hottest_artists_df = hottest_artists.toDF()
hottest_artists_df.createOrReplaceTempView("hottest_artists_df_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# %%local
# t_end = time.time()
# t_end - t_start

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
%%sql -o hottest_artists_df_tab
SELECT * FROM hottest_artists_df_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

## Number of Songs per aritst vs popular artists

In [10]:
songs_per_artist = table_body.map(lambda x : (x[column_index['get_artist_name']],
                                           [x[column_index['get_title']], 
                                           x[column_index['get_num_songs']],
                                           x[column_index['get_artist_hotttnesss']],
                                              x[column_index['get_duration']],
                                              x[column_index['get_loudness']]])).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
## Ineffecient code

# No_of_songs = songs_per_artist.filter(lambda x: x[1][1] != ''
#                 ).map(lambda x: (x[0], int(x[1][1]))
#                 ).groupByKey().mapValues(
#     lambda x: sum(x)).sortBy(
#     lambda x :x[1], ascending = False).map(lambda x: [x[0], x[1]]).filter(lambda x: x[1]>=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
No_of_songs = songs_per_artist.filter(lambda x: x[1][1] != ''
                ).map(lambda x: (x[0], int(x[1][1]))
                ).reduceByKey(lambda x,y: x+y).sortBy(
                 lambda x :x[1], ascending = False, numPartitions = ct_partitions)\
                 .map(lambda x: [x[0], x[1]]).filter(lambda x: x[1]>=10)

n_songs_df = No_of_songs.toDF()
n_songs_df.createOrReplaceTempView("n_songs_df_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
%%sql -o n_songs_df_tab
SELECT * FROM n_songs_df_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [14]:
%%local
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2, shared_yaxes=False, horizontal_spacing = 0.25,subplot_titles=[
        'Top 20 artists-Number of Songs', 
        'Top 20 artists-Popularity'
    ])

fig.add_trace(go.Bar(x=n_songs_df_tab['_2'][:20][::-1], y=n_songs_df_tab['_1'][:20][::-1], orientation='h'),
                    #marker=dict(color=n_songs_df['artist'], coloraxis="coloraxis")),
              1, 1)

fig.add_trace(go.Bar(x=hottest_artists_df_tab['_2'][:20][::-1], y=hottest_artists_df_tab['_1'][:20][::-1], orientation='h'),
                    #marker=dict(color=[2, 3, 5], coloraxis="coloraxis")),
              1, 2)

fig.update_layout(coloraxis=dict(colorscale='darkmint'), showlegend=False)

fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=30,
        t=40,
        pad=4
    )
    #paper_bgcolor="LightSteelBlue",
)
fig.update_yaxes(type='category')
fig.show()

# fig.write_image("fig1.png")
# fig.write_html("fig1.html")

## Artist Metrics

In [15]:
artist_keys = table_body.map(lambda x : (x[column_index['get_artist_name']], x[column_index['get_key']]))
artist_tempo = table_body.map(lambda x : (x[column_index['get_artist_name']], x[column_index['get_tempo']]))
artist_loudness = table_body.map(lambda x : (x[column_index['get_artist_name']], x[column_index['get_loudness']]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
artist_keys_agg = artist_keys.filter(lambda x: x[1] != '')\
                             .map(lambda x : (x[0], float(x[1])))\
                             .mapValues(lambda x: (x, 1))\
                             .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]),numPartitions = ct_partitions)\
                             .mapValues(lambda x: x[0]/x[1])

artist_tempo_agg = artist_tempo.filter(lambda x: x[1] != '')\
                               .map(lambda x : (x[0], float(x[1])))\
                               .mapValues(lambda x: (x, 1))\
                               .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]),numPartitions = ct_partitions)\
                               .mapValues(lambda x: x[0]/x[1])

artist_loudness_agg = artist_loudness.filter(lambda x: x[1] != '')\
                                     .map(lambda x : (x[0], float(x[1])))\
                                     .mapValues(lambda x: (x, 1))\
                                     .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]),numPartitions = ct_partitions)\
                                     .mapValues(lambda x: x[0]/x[1])\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
artist_metrics = artist_keys_agg.join(artist_tempo_agg)
all_metrics = artist_metrics.join(artist_loudness_agg, numPartitions = ct_partitions).map(lambda x : 
                                [x[0], x[1][0][0], x[1][0][1], x[1][1]]).filter(
                                lambda x : x[0] in hottest_artists_name[:20])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
all_metrics_df = all_metrics.toDF()
all_metrics_df.createOrReplaceTempView("all_metrics_df_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
%%sql -o all_metrics_df_tab
SELECT * FROM all_metrics_df_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [20]:
%%local
fig = px.scatter(all_metrics_df_tab, x='_3', 
                 y='_4',
                 
                 #color = '_1',
                 text = '_1',

                 labels={
                     "_1": "artist name",
                     "_2": "keys",
                     "_3": "tempo",
                     "_4": "loudness"
                     
                 },
                 color_discrete_sequence=px.colors.qualitative.D3_r,
                 hover_data=['_1'], log_x=False, size_max=60)


fig.update_traces(textposition='top center')

fig.update_traces(marker=dict(size=10,
                              line=dict(width=2
                                        )),
                  selector=dict(mode='markers'))

fig.update_layout(
    height=600,
    width=1000,
    title_text='Sound quality analysis of Popular artists'
    
)


fig.show()

# fig.write_image("fig3.png")
# fig.write_html("fig3.html")

## Popular Songs of popular artists

In [21]:
hottest_songs_rdd = table_body.map(lambda x : (x[column_index['get_artist_name']],
                                           x[column_index['get_title']], 
                                           x[column_index['get_song_hotttnesss']],
                                           x[column_index['get_artist_hotttnesss']],
                                              x[column_index['get_duration']],
                                              x[column_index['get_loudness']])).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
hottest_songs = hottest_songs_rdd.filter(lambda x : x[2] != ''
                     and x[3] != '' and x[4] != '' and x[5] != '').map(lambda x: (x[0],x[1],float(x[2]), 
                    float(x[3]), float(x[4]), float(x[5]))).sortBy(lambda x : x[3], 
                    ascending=False, numPartitions = ct_partitions).map(lambda x : [x[0], x[1].split('(')[0], x[2], x[3], x[4],
                                                     x[5]]).filter(lambda x: x[2]>0.97)#.filter(lambda x: x[0] 
                                                    #in hottest_artists_name[:30])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
hottest_songs_pop_artists = hottest_songs.filter(lambda x: x[0] in hottest_artists_name[:20])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
hottest_songs_df = hottest_songs.toDF()
hottest_songs_df.createOrReplaceTempView("hottest_songs_df_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
%%sql -o hottest_songs_df_tab
SELECT * FROM hottest_songs_df_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [26]:
hottest_songs_pop_df = hottest_songs_pop_artists.toDF()
hottest_songs_pop_df.createOrReplaceTempView("hottest_songs_pop_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
%%sql -o hottest_songs_pop_tab
SELECT * FROM hottest_songs_pop_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [28]:
%%local
import plotly.express as px

fig = px.scatter(hottest_songs_df_tab, x='_3', 
                 y='_6',
                 size =  '_5',
                 color = '_1',
                 text = '_2',
                 
                 labels={
                     "_4": "artist popularity",
                     "_3": "song hotness",
                     "_1": "Artist",
                     "_6": "loudness",
                     "_2": "song"
                     
                 },
                 color_discrete_sequence=px.colors.qualitative.Bold,
                  hover_data = ['_2'], log_x=False, size_max=30)
fig.update_traces(textposition='bottom center')

fig.update_layout(
    height=650,
    width=900,
    title_text='Popular Songs of All Time'
    
)
fig.show()


## Similar Artists

In [29]:
similar_artists = table_body.map(lambda x: (x[column_index['get_artist_id']],
                          [x[column_index['get_artist_name']], 
                           x[column_index['get_similar_artists']]]))

artists = similar_artists.map(lambda x :(x[0], x[1][0]))
ArtistDict = artists.collectAsMap()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
def get_list_of_artists(str_lst):
    str_lst = str_lst.replace('[', '').replace(']', '').replace("'", "")
    ids = str_lst.split()
    id_names = []
    for i in ids:
        if i in ArtistDict:
            id_names.append(ArtistDict[i])
    return id_names

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
similar_artists_names = similar_artists.map(lambda x: (x[1][0], get_list_of_artists(x[1][1])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
similar_artists_names_df = similar_artists_names.toDF()
similar_artists_names_df.createOrReplaceTempView("similar_artists_df_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
%%sql -o similar_artists_df_tab
SELECT * FROM similar_artists_df_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [36]:
%%local
t_end = time.time()
t_end - t_start

58.12773609161377

In [34]:
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Time Taken

### Initial time : 203 secs
### After minimizing actions : 180 secs 
### replacing groupbykey by reducebykey : 95 secs
### After adding numPartitions : 58 secs
